In [1]:
import pickle
from qiskit_ibm_provider import IBMProvider
import pandas as pd
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import qiskit_aer.noise as noise
from qiskit.quantum_info import hellinger_fidelity
from evaluation_metrics import TV
import numpy as np
from math import pow
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side

In [2]:
api_file = "../../ibm_API_key"
with open(api_file, "r") as f:
    api_key = f.readline().strip()
provider = IBMProvider(api_key, instance='ibm-q-ncsu/nc-state/quantum-compiler')

### Ideal results

In [3]:
with open('results/ideal_results.pkl', 'rb') as f:
    ideal_results = pickle.load(f)

In [7]:
def simulate(circ, backend, with_noise):
    if with_noise:
        # load noise model
        with open(f'noise_models/{backend.name}_noise_model.pkl', 'rb') as f:
            noise_model = pickle.load(f)

        # create a simulator for backend with noise model
        basis_gates = noise_model.basis_gates
        coupling_map = backend.configuration().coupling_map

        sim = AerSimulator(noise_model=noise_model,
                        basis_gates=basis_gates,
                        coupling_map=coupling_map)
    else:
        # create a simulator for backend
        sim = AerSimulator()

    # simulate and extract results
    shots = 4096
    simulator_result = sim.run(circ, shots = shots, seed_simulator=12345).result()
    simulator_counts = simulator_result.get_counts()

    #Reverse the key strings
    simulator_counts = {k[::-1]: v/shots for k, v in simulator_counts.items()}

    return simulator_counts

#ZNE function
def zne(dists,degree, number_of_points, top=65,percentage_mode=False):

    if percentage_mode:
        number_of_keys = (len(dists[0].keys()) * top) // 100
    else:
        number_of_keys = top

    #Getting the top 10 results from the distribution
    sorted_dist = dict(sorted(dists[0].items(), key=lambda x: x[1], reverse=True))
    top_k = list(sorted_dist.keys())[:number_of_keys]
    remaining_keys = list(sorted_dist.keys())[number_of_keys:]

    #Since this was global folding
    x_axis = np.array([i for i in range(1,number_of_points * 2,2)])

    #Extrapolation of top 10 results
    mitigated_top_k_dist = {}

    for key in top_k:
        mitigated_top_k_dist[key] = 0
        y_axis = np.array([ dists[i][key] if key in dists[i] else 0 for i in range(0,number_of_points)])
        coefficients = np.polyfit(x_axis, y_axis, degree)
        poly = np.poly1d(coefficients)
        mitigated_top_k_dist[key] = 0 if poly(0) < 0 else poly(0)

    #Calculate the sum of the mitigated top 10 distribution
    sum_mitigated_top_k_dist = sum(mitigated_top_k_dist.values())
    from functools import reduce
    sum_remaining_keys = reduce(lambda acc, key_value: acc + key_value[1] if key_value[0] in remaining_keys else acc, sorted_dist.items(), 0)
    total_sum = sum_mitigated_top_k_dist + sum_remaining_keys

    #Normalize the mitigated top 10 distribution
    normalized_mitigated_top_k_dist = dict(map(lambda x: (x[0],x[1]/total_sum),mitigated_top_k_dist.items()))
    normalized_mitigated_top_k_dist.update(dict(map(lambda x: (x[0],x[1]/total_sum),list(filter(lambda x: x[0] in remaining_keys,sorted_dist.items())))))
    return normalized_mitigated_top_k_dist

#Function for performing folding free ZNE
def folding_free_zne(dist,reliability,N,top=65,percentage_mode=False):

    if percentage_mode:
        number_of_keys = (len(dist.keys()) * top) // 100
    else:
        number_of_keys = top

    #Getting the top 10 results from the distribution
    sorted_dist = dict(sorted(dist.items(), key=lambda x: x[1], reverse=True))
    top_k = list(sorted_dist.keys())[:number_of_keys]
    remaining_keys = list(sorted_dist.keys())[number_of_keys:]

    x_axis = [(1-reliability),1]

    #Extrapolation of top k results
    mitigated_top_k_dist = {}

    for key in top_k:
        mitigated_top_k_dist[key] = 0
        y_axis = [dist[key] if key in dist else 0,pow(2,-N)]
        coefficients = np.polyfit(x_axis, y_axis, 1)
        poly = np.poly1d(coefficients)
        mitigated_top_k_dist[key] = 0 if poly(0) < 0 else poly(0)

    #Calculate the sum of the mitigated top 10 distribution
    sum_mitigated_top_k_dist = sum(mitigated_top_k_dist.values())
    from functools import reduce
    sum_remaining_keys = reduce(lambda acc, key_value: acc + key_value[1] if key_value[0] in remaining_keys else acc, sorted_dist.items(), 0)
    total_sum = sum_mitigated_top_k_dist + sum_remaining_keys

    #Normalize the mitigated top 10 distribution
    normalized_mitigated_top_k_dist = dict(map(lambda x: (x[0],x[1]/total_sum),mitigated_top_k_dist.items()))
    normalized_mitigated_top_k_dist.update(dict(map(lambda x: (x[0],x[1]/total_sum),list(filter(lambda x: x[0] in remaining_keys,sorted_dist.items())))))

    return normalized_mitigated_top_k_dist

#Function for performing reliability based ZNE
def reliability_based_zne(dists,N,reliabilities,number_of_points,top=65,percentage_mode=False):

    if percentage_mode:
        number_of_keys = (len(dists[0].keys()) * top) // 100
    else:
        number_of_keys = top

    #Getting the top 10 results from the distribution
    sorted_dist = dict(sorted(dists[0].items(), key=lambda x: x[1], reverse=True))
    top_k = list(sorted_dist.keys())[:number_of_keys]
    remaining_keys = list(sorted_dist.keys())[number_of_keys:]

    x_axis = list(map(lambda x: 1-x, reliabilities))
    #x_axis = x_axis[:number_of_points] + [1]
    x_axis = x_axis[:number_of_points]
    
    #Extrapolation of top 10 results
    mitigated_top_k_dist = {}

    for key in top_k:
        mitigated_top_k_dist[key] = 0
        #y_axis = np.array([dists[i][key] if key in dists[i] else 0 for i in range(0,number_of_points)] + [pow(2,-N)]) 
        y_axis = np.array([dists[i][key] if key in dists[i] else 0 for i in range(0,number_of_points)])
        coefficients = np.polyfit(x_axis, y_axis, 1)
        poly = np.poly1d(coefficients)
        mitigated_top_k_dist[key] = 0 if poly(0) < 0 else poly(0)

    #Calculate the sum of the mitigated top 10 distribution
    sum_mitigated_top_k_dist = sum(mitigated_top_k_dist.values())
    from functools import reduce
    sum_remaining_keys = reduce(lambda acc, key_value: acc + key_value[1] if key_value[0] in remaining_keys else acc, sorted_dist.items(), 0)
    total_sum = sum_mitigated_top_k_dist + sum_remaining_keys

    #Normalize the mitigated top 10 distribution
    normalized_mitigated_top_10_dist = dict(map(lambda x: (x[0],x[1]/total_sum),mitigated_top_k_dist.items()))
    normalized_mitigated_top_10_dist.update(dict(map(lambda x: (x[0],x[1]/total_sum),list(filter(lambda x: x[0] in remaining_keys,sorted_dist.items())))))

    return normalized_mitigated_top_10_dist

In [8]:
summary_dict_list = []

for system in ['ibm_brisbane','ibm_kyoto','ibm_sherbrooke','ibm_nazca']:

    backend = provider.get_backend(system)

    for N in [4,6,8,10]:
        for T in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.5,2.0]:

            print(f'Processing {system} N={N} T={T}')

            # Load the circuit data
            with open(f'circuits/{system}_{N}_{str(T).replace('.','-')}.pkl', 'rb') as f:
                circuit_data = pickle.load(f)

            # Get the circuit data
            layout = circuit_data['layout']
            circuits = circuit_data['circuit']
            reliabilities = circuit_data['circuit_reliabilities']

            # Get the simulation distributions
            ideal_result = ideal_results[N,T]
            without_noise_simulation_result = simulate(circuits[0], backend, with_noise=False)
            noise_simulation_result = list(map(lambda circ: simulate(circ, backend, with_noise=True), circuits))

            # Calculate the Hellinger fidelity
            without_noise_simulation_hf = hellinger_fidelity(ideal_result, without_noise_simulation_result)
            noise_simulation_hf = list(map(lambda result: hellinger_fidelity(ideal_result, result), noise_simulation_result))

            # Calculate the TV distance
            without_noise_simulation_tv = TV(ideal_result, without_noise_simulation_result)
            noise_simulation_tv = list(map(lambda result: TV(ideal_result, result), noise_simulation_result))

            # Get the complete set of keys
            keys = set(ideal_result.keys()) | \
                    set(without_noise_simulation_result.keys()) | \
                    set(noise_simulation_result[0].keys()) | \
                    set(noise_simulation_result[1].keys()) | \
                    set(noise_simulation_result[2].keys()) | \
                    set(noise_simulation_result[3].keys()) | \
                    set(noise_simulation_result[4].keys())

            keys = sorted(list(keys))

            #Create a df and assign keys as index and name the column as State and add the values of keys to that column
            distribution_data_df = pd.DataFrame(keys,columns=['State'],index=keys)

            #map the results to the distribution data
            distribution_data_df['Ideal distribution'] = distribution_data_df['State'].map(ideal_result)
            distribution_data_df['Without Noise Distribution'] = distribution_data_df['State'].map(without_noise_simulation_result)
            distribution_data_df['Noise Distribution (Factor 1)'] = distribution_data_df['State'].map(noise_simulation_result[0])
            distribution_data_df['Noise Distribution (Factor 3)'] = distribution_data_df['State'].map(noise_simulation_result[1])
            distribution_data_df['Noise Distribution (Factor 5)'] = distribution_data_df['State'].map(noise_simulation_result[2])
            distribution_data_df['Noise Distribution (Factor 7)'] = distribution_data_df['State'].map(noise_simulation_result[3])
            distribution_data_df['Noise Distribution (Factor 9)'] = distribution_data_df['State'].map(noise_simulation_result[4])

            #ZNE
            zne_results = zne(noise_simulation_result,2,3,10)
            zne_hf = hellinger_fidelity(ideal_result, zne_results)
            zne_tv = TV(ideal_result, zne_results)
            distribution_data_df['ZNE distribution'] = distribution_data_df['State'].map(zne_results)

            #FFZNE
            ffzne_results = folding_free_zne(noise_simulation_result[0],reliabilities[0],N,10)
            ffzne_hf = hellinger_fidelity(ideal_result, ffzne_results)
            ffzne_tv = TV(ideal_result, ffzne_results)
            distribution_data_df['FFZNE distribution'] = distribution_data_df['State'].map(ffzne_results)

            #RBZNE
            rbzne_results = reliability_based_zne(noise_simulation_result,N,reliabilities,3,10)
            rbzne_hf = hellinger_fidelity(ideal_result, rbzne_results)
            rbzne_tv = TV(ideal_result, rbzne_results)
            distribution_data_df['RBZNE distribution'] = distribution_data_df['State'].map(rbzne_results)

            #Sort based on Simulation probability
            distribution_data_df = distribution_data_df.sort_values(by='Ideal distribution',ascending=False)

            summary_dict_list.append({  'N': N,
                                        'T': T,
                                        'System': system,
                                        'Without Noise HF': without_noise_simulation_hf,
                                        'Without Noise TV': without_noise_simulation_tv,
                                        'Noise HF': noise_simulation_hf[0],
                                        'Noise TV': noise_simulation_tv[0],
                                        'ZNE HF': zne_hf,
                                        'ZNE TV': zne_tv,
                                        'FFZNE HF': ffzne_hf,
                                        'FFZNE TV': ffzne_tv,
                                        'RBZNE HF': rbzne_hf,
                                        'RBZNE TV': rbzne_tv
                                    })

            counter = int(pow(2,N) + 3)
            sheet_name = 'N_{}_T_{}'.format(N,T)

            #write in append mode
            with pd.ExcelWriter("results/{}_distribution_data.xlsx".format(system),mode='a', engine='openpyxl') as writer:
                distribution_data_df.to_excel(writer, sheet_name=sheet_name, index=False)
                workbook = writer.book
                worksheet = writer.sheets[sheet_name]
                
                # Accessing the active sheet
                active_sheet = workbook.active
                
                # Write to cell A19
                worksheet['A{}'.format(str(counter))] = 'Hellinger fidelity'
                worksheet['A{}'.format(str(counter+1))] = 'TV'
                worksheet['C{}'.format(str(counter))] = without_noise_simulation_hf
                worksheet['C{}'.format(str(counter+1))] = without_noise_simulation_tv
                worksheet['D{}'.format(str(counter))] = noise_simulation_hf[0]
                worksheet['D{}'.format(str(counter+1))] = noise_simulation_tv[0]
                worksheet['E{}'.format(str(counter))] = noise_simulation_hf[1]
                worksheet['E{}'.format(str(counter+1))] = noise_simulation_tv[1]
                worksheet['F{}'.format(str(counter))] = noise_simulation_hf[2]
                worksheet['F{}'.format(str(counter+1))] = noise_simulation_tv[2]
                worksheet['G{}'.format(str(counter))] = noise_simulation_hf[3]
                worksheet['G{}'.format(str(counter+1))] = noise_simulation_tv[3]
                worksheet['H{}'.format(str(counter))] = noise_simulation_hf[4]
                worksheet['H{}'.format(str(counter+1))] = noise_simulation_tv[4]
                worksheet['I{}'.format(str(counter))] = zne_hf
                worksheet['I{}'.format(str(counter+1))] = zne_tv
                worksheet['J{}'.format(str(counter))] = ffzne_hf
                worksheet['J{}'.format(str(counter+1))] = ffzne_tv
                worksheet['K{}'.format(str(counter))] = rbzne_hf
                worksheet['K{}'.format(str(counter+1))] = rbzne_tv

                worksheet['A{}'.format(str(counter+2))] = 'Circuit reliabilities'
                worksheet['D{}'.format(str(counter+2))] = reliabilities[0]
                worksheet['E{}'.format(str(counter+2))] = reliabilities[1]
                worksheet['F{}'.format(str(counter+2))] = reliabilities[2]
                worksheet['G{}'.format(str(counter+2))] = reliabilities[3]
                worksheet['H{}'.format(str(counter+2))] = reliabilities[4]

                worksheet['A{}'.format(str(counter+3))] = 'Circuit depth'
                worksheet['D{}'.format(str(counter+3))] = circuits[0].depth()
                worksheet['E{}'.format(str(counter+3))] = circuits[1].depth()
                worksheet['F{}'.format(str(counter+3))] = circuits[2].depth()
                worksheet['G{}'.format(str(counter+3))] = circuits[3].depth()
                worksheet['H{}'.format(str(counter+3))] = circuits[4].depth()

                worksheet['A{}'.format(str(counter+4))] = 'RZX count'
                worksheet['D{}'.format(str(counter+4))] = circuits[0].count_ops()['rzx']
                worksheet['E{}'.format(str(counter+4))] = circuits[1].count_ops()['rzx']
                worksheet['F{}'.format(str(counter+4))] = circuits[2].count_ops()['rzx']
                worksheet['G{}'.format(str(counter+4))] = circuits[3].count_ops()['rzx']
                worksheet['H{}'.format(str(counter+4))] = circuits[4].count_ops()['rzx']

                worksheet['A{}'.format(str(counter+5))] = 'RX count'
                worksheet['D{}'.format(str(counter+5))] = circuits[0].count_ops()['rx']
                worksheet['E{}'.format(str(counter+5))] = circuits[1].count_ops()['rx']
                worksheet['F{}'.format(str(counter+5))] = circuits[2].count_ops()['rx']
                worksheet['G{}'.format(str(counter+5))] = circuits[3].count_ops()['rx']
                worksheet['H{}'.format(str(counter+5))] = circuits[4].count_ops()['rx']

                #Create a border around A19 to J21
                border = Border(left=Side(border_style='thin', color='FF000000'),
                                right=Side(border_style='thin', color='FF000000'),
                                top=Side(border_style='thin', color='FF000000'),
                                bottom=Side(border_style='thin', color='FF000000'))
                
                left_thick_border = Border(left=Side(border_style='medium', color='FF000000'))
                bottom_thick_border = Border(bottom=Side(border_style='medium', color='FF000000'))
                right_thick_border = Border(right=Side(border_style='medium', color='FF000000'))

                heading_border = Border(left=Side(border_style='medium', color='FF000000'),
                                        right=Side(border_style='medium', color='FF000000'),
                                        top=Side(border_style='medium', color='FF000000'),
                                        bottom=Side(border_style='medium', color='FF000000'))
                
                bottom_right_thick_border = Border(right=Side(border_style='medium', color='FF000000'),
                                                    bottom=Side(border_style='medium', color='FF000000'))

                for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=1, max_col=1):
                    for cell in row:
                        cell.border = left_thick_border

                for row in worksheet.iter_rows(min_row=1, max_row=counter-2, min_col=11, max_col=11):
                    for cell in row:
                        cell.border = right_thick_border

                for row in worksheet.iter_rows(min_row=counter-2, max_row=counter-2, min_col=1, max_col=11):
                    for cell in row:
                        cell.border = bottom_thick_border
                
                for row in worksheet.iter_rows(min_row=counter, max_row=counter+5, min_col=1, max_col=11):
                    for cell in row:
                        cell.border = border

                for row in worksheet.iter_rows(min_row=1, max_row=1, min_col=1, max_col=11):
                    for cell in row:
                        cell.border = heading_border

                #assign bottom right thick border to J19
                worksheet['K{}'.format(str(counter-2))].border = bottom_right_thick_border

                # Auto-adjusting column widths
                for column_cells in worksheet.columns:
                    length = max(len(str(cell.value)) for cell in column_cells)
                    worksheet.column_dimensions[column_cells[0].column_letter].width = length

                # Centering text in every cell
                for row in worksheet.iter_rows():
                    for cell in row:
                        cell.alignment = Alignment(horizontal='center', vertical='center')

Processing ibm_brisbane N=4 T=0.1


C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\

Processing ibm_brisbane N=4 T=0.2


C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\

Processing ibm_brisbane N=4 T=0.3


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\

Processing ibm_brisbane N=4 T=0.9


C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29064\780643242.py:86: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(x_axis, y_axis, 2)
C:\Users\Admin\AppData\Local\Temp\

Processing ibm_brisbane N=4 T=1.0


KeyboardInterrupt: 

In [6]:
summary_df = pd.DataFrame(summary_dict_list)
summary_df['ZNE HF % change'] = ((summary_df['ZNE HF'] - summary_df['Noise HF']) / summary_df['Noise HF']) * 100
summary_df['ZNE TV % change'] = ((summary_df['Noise TV'] - summary_df['ZNE TV']) / summary_df['Noise TV']) * 100
summary_df['FFZNE HF % change'] = ((summary_df['FFZNE HF'] - summary_df['Noise HF']) / summary_df['Noise HF']) * 100
summary_df['FFZNE TV % change'] = ((summary_df['Noise TV'] - summary_df['FFZNE TV']) / summary_df['Noise TV']) * 100
summary_df['RBZNE HF % change'] = ((summary_df['RBZNE HF'] - summary_df['Noise HF']) / summary_df['Noise HF']) * 100
summary_df['RBZNE TV % change'] = ((summary_df['Noise TV'] - summary_df['RBZNE TV']) / summary_df['Noise TV']) * 100

#Arrange the columns
summary_df = summary_df[['N', 'T', 'System', 
                         'Without Noise HF', 'Noise HF', 
                            'ZNE HF', 'ZNE HF % change',
                            'FFZNE HF', 'FFZNE HF % change',
                            'RBZNE HF', 'RBZNE HF % change',
                            'Without Noise TV', 'Noise TV',
                            'ZNE TV', 'ZNE TV % change',
                            'FFZNE TV', 'FFZNE TV % change',
                            'RBZNE TV', 'RBZNE TV % change']]

#Sort these records by System, N and T
summary_df = summary_df.sort_values(by=['System', 'N', 'T'])

with pd.ExcelWriter("results/summary_distribution_data.xlsx".format(system), engine='openpyxl') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Summary']
    
    # Accessing the active sheet
    active_sheet = workbook.active

    # Calculate Geometric mean of 1 + (% change / 100) for Hellinger fidelity and TV
    worksheet['A213'] = 'Geometric mean'
    worksheet['G213'] = (1 + summary_df['ZNE HF % change'] / 100).prod() ** (1 / len(summary_df))
    worksheet['O213'] = (1 + summary_df['ZNE TV % change'] / 100).prod() ** (1 / len(summary_df))
    worksheet['I213'] = (1 + summary_df['FFZNE HF % change'] / 100).prod() ** (1 / len(summary_df))
    worksheet['Q213'] = (1 + summary_df['FFZNE TV % change'] / 100).prod() ** (1 / len(summary_df))
    worksheet['K213'] = (1 + summary_df['RBZNE HF % change'] / 100).prod() ** (1 / len(summary_df))
    worksheet['S213'] = (1 + summary_df['RBZNE TV % change'] / 100).prod() ** (1 / len(summary_df))

    #Calculate Geometric mean of 1 + (% change / 100) for Hellinger fidelity and TV for each systems
    worksheet['A214'] = 'Geometric mean for ibm_kyoto'
    worksheet['G214'] = (1 + summary_df[summary_df['System'] == 'ibm_kyoto']['ZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_kyoto']))
    worksheet['O214'] = (1 + summary_df[summary_df['System'] == 'ibm_kyoto']['ZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_kyoto']))
    worksheet['I214'] = (1 + summary_df[summary_df['System'] == 'ibm_kyoto']['FFZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_kyoto']))
    worksheet['Q214'] = (1 + summary_df[summary_df['System'] == 'ibm_kyoto']['FFZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_kyoto']))
    worksheet['K214'] = (1 + summary_df[summary_df['System'] == 'ibm_kyoto']['RBZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_kyoto']))
    worksheet['S214'] = (1 + summary_df[summary_df['System'] == 'ibm_kyoto']['RBZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_kyoto']))

    worksheet['A215'] = 'Geometric mean for ibm_sherbrooke'
    worksheet['G215'] = (1 + summary_df[summary_df['System'] == 'ibm_sherbrooke']['ZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_sherbrooke']))
    worksheet['O215'] = (1 + summary_df[summary_df['System'] == 'ibm_sherbrooke']['ZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_sherbrooke']))
    worksheet['I215'] = (1 + summary_df[summary_df['System'] == 'ibm_sherbrooke']['FFZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_sherbrooke']))
    worksheet['Q215'] = (1 + summary_df[summary_df['System'] == 'ibm_sherbrooke']['FFZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_sherbrooke']))
    worksheet['K215'] = (1 + summary_df[summary_df['System'] == 'ibm_sherbrooke']['RBZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_sherbrooke']))
    worksheet['S215'] = (1 + summary_df[summary_df['System'] == 'ibm_sherbrooke']['RBZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_sherbrooke']))

    worksheet['A216'] = 'Geometric mean for ibm_nazca'
    worksheet['G216'] = (1 + summary_df[summary_df['System'] == 'ibm_nazca']['ZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_nazca']))
    worksheet['O216'] = (1 + summary_df[summary_df['System'] == 'ibm_nazca']['ZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_nazca']))
    worksheet['I216'] = (1 + summary_df[summary_df['System'] == 'ibm_nazca']['FFZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_nazca']))
    worksheet['Q216'] = (1 + summary_df[summary_df['System'] == 'ibm_nazca']['FFZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_nazca']))
    worksheet['K216'] = (1 + summary_df[summary_df['System'] == 'ibm_nazca']['RBZNE HF % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_nazca']))
    worksheet['S216'] = (1 + summary_df[summary_df['System'] == 'ibm_nazca']['RBZNE TV % change'] / 100).prod() ** (1 / len(summary_df[summary_df['System'] == 'ibm_nazca']))

    # Auto-adjusting column widths
    for column_cells in worksheet.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        worksheet.column_dimensions[column_cells[0].column_letter].width = length

    # Centering text in every cell
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center')
                            

In [116]:
list(map(lambda x: hellinger_fidelity(x, ideal_result), without_noise_simulation_result))

[0.9929843469726516,
 0.9929843469726516,
 0.9929843469726516,
 0.9929843469726516,
 0.9929843469726516]

In [117]:
list(map(lambda x: hellinger_fidelity(x, ideal_result), noise_simulation_result))

[0.9677234582092556,
 0.9309184744481795,
 0.9044210928651266,
 0.8772649230512174,
 0.8564561388984665]

### Simulator result

In [23]:
import numpy as np
import pickle
from hamiltonian_models import Ising

N = 4
T = 3

h = np.array([1 for j in range(N)])
J_chain = np.zeros((N, N))
for j in range(N - 1):
    J_chain[j, j + 1] = 1

J_cycle = np.copy(J_chain)
J_cycle[0, N - 1] = 1

#On the IBM devices, only Ising_chain has a solution, therefore keeping it
print("Creating the model with {} qubits and simulation time to be: {} units".format(N,T))
Ising_chain = Ising(N, T, J_chain, h)

Creating the model with 4 qubits and simulation time to be: 3 units


In [24]:
from ibm_provider_custom import IBMProvider as IBMProviderCustom
ibm = IBMProviderCustom(from_file="../ibm_API_key", hub="ibm-q-ncsu", group="nc-state", project="quantum-compiler")

In [25]:
ibm.compile(Ising_chain, backend=system, trotter_num=4,use_pulse=True)

[0, 1, 2, 3]


In [26]:
circuit = ibm.prog

In [27]:
len(circuit)

5

In [ ]:
circuit[0].draw(output='mpl')

In [28]:
#Get the backend
backend = provider.get_backend('ibm_brisbane')

circ = circuit[0]

In [29]:
ideal_result = ideal_results[N,T]

### Noise model creation

In [93]:
noise_model = NoiseModel.from_backend(backend)

# Get the rzx errors
with open('error_rates.pkl', 'rb') as f:
    error_rates = pickle.load(f)

# Add rzx errors
for qubit, error in error_rates[system]['rzx'].items():
    noise_model.add_quantum_error(noise.depolarizing_error(error,2),"rzx",list(qubit))

# Add rx errors
for qubit, error in error_rates[system]['rx'].items():
    noise_model.add_quantum_error(noise.depolarizing_error(error,1),"rx",[qubit])

In [96]:
# create a simulator for backend
basis_gates = noise_model.basis_gates
coupling_map = backend.configuration().coupling_map

sim = AerSimulator(noise_model=noise_model,
                   basis_gates=basis_gates,
                   coupling_map=coupling_map)

# simulate and extract results
simulator_result = sim.run(circ, shots = 4096, seed_simulator=12345).result()
simulator_counts = simulator_result.get_counts()

#Reverse the key string
simulator_counts = {k[::-1]: v/4096 for k, v in simulator_counts.items()}

#Calculate hellinger fidelity
from qiskit.quantum_info import hellinger_fidelity
hellinger_fidelity(ideal_result, simulator_counts)

0.9680807730817306

### Default Noise model

In [97]:
# create a simulator for backend
noise_model = NoiseModel.from_backend(backend)
basis_gates = noise_model.basis_gates
coupling_map = backend.configuration().coupling_map

sim = AerSimulator(noise_model=noise_model,
                   basis_gates=basis_gates,
                   coupling_map=coupling_map)

# simulate and extract results
simulator_result = sim.run(circ, shots = 4096, seed_simulator=12345).result()
simulator_counts = simulator_result.get_counts()

#Reverse the key string
simulator_counts = {k[::-1]: v/4096 for k, v in simulator_counts.items()}

#Calculate hellinger fidelity
from qiskit.quantum_info import hellinger_fidelity
hellinger_fidelity(ideal_result, simulator_counts)

0.985270406474338

### Without noise simulator

In [30]:
# create a simulator for backend
sim = AerSimulator()

# simulate and extract results
simulator_result = sim.run(circ, shots = 4096, seed_simulator=12345).result()
simulator_counts = simulator_result.get_counts()

#Reverse the key string
simulator_counts = {k[::-1]: v/4096 for k, v in simulator_counts.items()}

#Calculate hellinger fidelity
from qiskit.quantum_info import hellinger_fidelity
hellinger_fidelity(ideal_result, simulator_counts)

0.35439788695954233